In [280]:
import numpy as np
import pymorphy2
import pandas as pd
from tqdm.notebook import tqdm
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [263]:
data_original=open('dogs.txt').readlines()
data_original

['Мистер Шерлок Холмс, имевший обыкновение вставать очень поздно, за исключением тех нередких случаев, когда вовсе не ложился спать, сидел за завтраком. Я стоял на коврике перед камином и держал в руках трость, которую наш посетитель забыл накануне вечером. Это была красивая, толстая палка с круглым набалдашником. Как раз под ним палку обхватывала широкая (в дюйм ширины) серебряная лента, а на этой ленте было выгравировано: «Джэмсу Мортимеру, M. R. С. S. от его друзей из С. С. Н.» и год «1884». Это была как раз такого рода трость, какую носят обыкновенно старомодные семейные доктора, — почтенная, прочная и надежная.\n',
 '\n',
 '— Что вы с нею делаете, Ватсон?\n',
 '\n',
 'Холмс сидел ко мне спиной, а я ничем не обнаружил своего занятия.\n',
 '\n',
 '— Почему вы узнали, что я делаю? У вас, должно быть, есть глаза в затылке.\n',
 '\n',
 '— У меня по крайней мере есть хорошо отполированный кофейник, и он стоит передо мною, — ответил он. — Но скажите мне, Ватсон, что вы делаете с тростью 

In [264]:
data=[line.replace('\n',' ') for line in data_original]
data=[line.replace('.',' . ') for line in data]
data=[line.replace(',',' ,') for line in data]
data=[line.replace('!',' !') for line in data]
data=[line.replace('?',' ?') for line in data]
data=[line.replace(';',' ;') for line in data]
data=[line.replace(':',' :') for line in data]
data=[line.replace('«','« ') for line in data]
data=[line.replace('»',' »') for line in data]
data=[line.replace('“',' “') for line in data]
data=[line.replace('„','„ ') for line in data]
data=[line.replace('(','( ') for line in data]
data=[line.replace(')',' )') for line in data]
data

['Мистер Шерлок Холмс , имевший обыкновение вставать очень поздно , за исключением тех нередких случаев , когда вовсе не ложился спать , сидел за завтраком .  Я стоял на коврике перед камином и держал в руках трость , которую наш посетитель забыл накануне вечером .  Это была красивая , толстая палка с круглым набалдашником .  Как раз под ним палку обхватывала широкая ( в дюйм ширины ) серебряная лента , а на этой ленте было выгравировано : « Джэмсу Мортимеру , M .  R .  С .  S .  от его друзей из С .  С .  Н .  » и год « 1884 » .  Это была как раз такого рода трость , какую носят обыкновенно старомодные семейные доктора , — почтенная , прочная и надежная .  ',
 ' ',
 '— Что вы с нею делаете , Ватсон ? ',
 ' ',
 'Холмс сидел ко мне спиной , а я ничем не обнаружил своего занятия .  ',
 ' ',
 '— Почему вы узнали , что я делаю ? У вас , должно быть , есть глаза в затылке .  ',
 ' ',
 '— У меня по крайней мере есть хорошо отполированный кофейник , и он стоит передо мною , — ответил он .  — 

In [265]:
sentences=[]
for line in data:
    sentences.extend(line.split('.'))

sentences=[sent for sent in sentences if sent!='  ']

tokens=[sent.split(' ') for sent in sentences]

tokens_tmp=[]
N=0
for i,tokens_list in enumerate(tokens):
    tokens_tmp.append([])
    for token in tokens_list:
        if token!='':
            tokens_tmp[i].append(token)
            N+=1

tokens=tokens_tmp
print(N)
tokens

24394


[['Мистер',
  'Шерлок',
  'Холмс',
  ',',
  'имевший',
  'обыкновение',
  'вставать',
  'очень',
  'поздно',
  ',',
  'за',
  'исключением',
  'тех',
  'нередких',
  'случаев',
  ',',
  'когда',
  'вовсе',
  'не',
  'ложился',
  'спать',
  ',',
  'сидел',
  'за',
  'завтраком'],
 ['Я',
  'стоял',
  'на',
  'коврике',
  'перед',
  'камином',
  'и',
  'держал',
  'в',
  'руках',
  'трость',
  ',',
  'которую',
  'наш',
  'посетитель',
  'забыл',
  'накануне',
  'вечером'],
 ['Это',
  'была',
  'красивая',
  ',',
  'толстая',
  'палка',
  'с',
  'круглым',
  'набалдашником'],
 ['Как',
  'раз',
  'под',
  'ним',
  'палку',
  'обхватывала',
  'широкая',
  '(',
  'в',
  'дюйм',
  'ширины',
  ')',
  'серебряная',
  'лента',
  ',',
  'а',
  'на',
  'этой',
  'ленте',
  'было',
  'выгравировано',
  ':',
  '«',
  'Джэмсу',
  'Мортимеру',
  ',',
  'M'],
 ['R'],
 ['С'],
 ['S'],
 ['от', 'его', 'друзей', 'из', 'С'],
 ['С'],
 ['Н'],
 ['»', 'и', 'год', '«', '1884', '»'],
 ['Это',
  'была',
  'как',
  

## Абсолютная частота

ADJF - NOUN

In [266]:
def get_pos_tokens(golden_pos,tokens):
    pos_list=[]
    for tokens_list in tokens:
        for token in tokens_list:
            token_pos=morph.parse(token)[0].tag.POS
            # if token_pos is not None:
            if token_pos is not None and golden_pos in token_pos:
                pos_list.append(morph.parse(token)[0].normal_form)

    return np.sort(list(set(pos_list)))

In [335]:
golden_pos_1='NOUN'
golden_pos_2='NOUN'

adj_list=get_pos_tokens(golden_pos_1,tokens)
noun_list=get_pos_tokens(golden_pos_2,tokens)

print(golden_pos_1,len(adj_list))
print(golden_pos_2,len(noun_list))

adj2idx=dict((adj,i) for i,adj in enumerate(adj_list))
noun2idx=dict((noun,i) for i,noun in enumerate(noun_list))

idx2adj=dict((i,adj) for i,adj in enumerate(adj_list))
idx2noun=dict((i,noun) for i,noun in enumerate(noun_list))

NOUN 1327
NOUN 1327


In [336]:
adj_noun_distr=np.zeros((len(adj_list),len(noun_list)))

for tokens_list in tokens:
    for j,token in enumerate(tokens_list[:-1]):
        token_1_parse=morph.parse(token)[0]
        token_2_parse=morph.parse(tokens_list[j+1])[0]
        # print(token_1_parse)
        if token_1_parse.tag.POS is not None and token_2_parse.tag.POS is not None:
            if golden_pos_1 in token_1_parse.tag.POS and golden_pos_2 in token_2_parse.tag.POS:
                row=adj2idx[token_1_parse.normal_form]
                column=noun2idx[token_2_parse.normal_form]
                adj_noun_distr[row,column]+=1


In [337]:
adj_noun_distr_norm=np.zeros((len(adj_list),len(noun_list)))

for i,line in enumerate(adj_noun_distr):
    if sum(line)!=0:
        adj_noun_distr_norm[i]=adj_noun_distr[i]/sum(line)

print(adj_noun_distr.sum())

760.0


In [338]:
df=pd.DataFrame({'adj':'None','noun':'noun','probab':[0],'cases':[0],'all cases':[0]})

for i,line in enumerate(adj_noun_distr_norm):
    if sum(line)!=0 :
        column_key=np.argmax(line)
        if adj_noun_distr[i,column_key]>2:
            adj=idx2adj[i]
            noun=idx2noun[column_key]
            df=df.append({'adj':adj,'noun':noun,'probab':round(line[column_key],3),'cases':adj_noun_distr[i,column_key],'all cases':adj_noun_distr[i].sum()},ignore_index=True)

/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/4113408072.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'probab':round(line[column_key],3),'cases':adj_noun_distr[i,column_key],'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/4113408072.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'probab':round(line[column_key],3),'cases':adj_noun_distr[i,column_key],'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/4113408072.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':a

In [340]:
df.sort_values('probab',ascending=False).head(20)

,adj,noun,probab,cases,all cases
2,гюго,баскервиль,1.000,7.0,7.0
5,мисс,стапльтон,1.000,6.0,6.0
9,струйка,дым,1.000,3.0,3.0
14,шерлок,холмс,1.000,22.0,22.0
8,смерть,сэр,0.923,12.0,13.0
1,генри,баскервиль,0.909,20.0,22.0
12,чарльз,баскервиль,0.875,7.0,8.0
11,телеграмма,мистер,0.750,3.0,4.0
4,доктор,мортимера,0.741,60.0,81.0
3,джэмс,мортимера,0.667,4.0,6.0


## Мера MI

In [329]:
df=pd.DataFrame({'adj':'None','noun':'noun','mi':[0],'cases':[0],'all cases':[0]})

for i,line in enumerate(tqdm(adj_noun_distr_norm)):
        for j,col in enumerate(line):
            num=adj_noun_distr_norm[i,j]*N
            den=adj_noun_distr_norm[i].sum()*adj_noun_distr_norm[:,j].sum()
            cases=adj_noun_distr[i,j]
            if den!=0 and num!=0 and cases>2:
                mi=np.log2(num/den)
                noun=idx2noun[j]
                adj=idx2adj[i]
                df=df.append({'adj':adj,'noun':noun,'mi':round(mi,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)

  0%|          | 0/682 [00:00<?, ?it/s]

/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2362909883.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'mi':round(mi,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2362909883.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'mi':round(mi,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2362909883.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'mi':round(mi,3),'cases':cases,'all cases':adj_noun_distr[i].sum

In [330]:
df.sort_values('mi',ascending=False).head(20)

,adj,noun,mi,cases,all cases
39,чёрный,борода,14.574,4.0,12.0
9,крайний,мера,14.499,7.0,9.0
10,милый,ватсон,14.446,3.0,11.0
34,тисовый,аллея,14.389,7.0,7.0
1,бедный,сэр,14.333,3.0,6.0
44,этот,трость,14.291,4.0,86.0
24,нортумберландский,отель,14.094,7.0,7.0
29,свой,желание,13.865,3.0,99.0
22,наш,приятель,13.700,3.0,60.0
26,передовой,статья,13.566,5.0,5.0


## Мера t-score

In [331]:
df=pd.DataFrame({'adj':'None','noun':'noun','t_score':[0],'cases':[0],'all cases':[0]})

for i,line in enumerate(tqdm(adj_noun_distr_norm)):
    for j,col in enumerate(line):
        num=adj_noun_distr_norm[i,j] - (adj_noun_distr_norm[i].sum()*adj_noun_distr_norm[:,j].sum())/N
        den=np.sqrt(adj_noun_distr_norm[i,j])
        cases=adj_noun_distr[i,j]
        if den!=0 and num!=0 and cases>2:
            t_score=num/den
            noun=idx2noun[j]
            adj=idx2adj[i]
            df=df.append({'adj':adj,'noun':noun,'t_score':round(t_score,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)

  0%|          | 0/682 [00:00<?, ?it/s]

/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2038486136.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'t_score':round(t_score,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2038486136.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'t_score':round(t_score,3),'cases':cases,'all cases':adj_noun_distr[i].sum()},ignore_index=True)
/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_93951/2038486136.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'adj':adj,'noun':noun,'t_score':round(t_score,3),'cases':cases,'al

In [332]:
df.sort_values('t_score',ascending=False).head(20)

,adj,noun,t_score,cases,all cases
24,нортумберландский,отель,1.000,7.0,7.0
26,передовой,статья,1.000,5.0,5.0
36,трагический,смерть,1.000,3.0,3.0
34,тисовый,аллея,1.000,7.0,7.0
8,коричневый,сапог,1.000,6.0,6.0
37,умный,человек,0.999,3.0,3.0
9,крайний,мера,0.882,7.0,9.0
4,гримпенский,трясина,0.866,3.0,4.0
25,опасный,место,0.866,3.0,4.0
5,деревенский,врач,0.816,4.0,6.0
